In [2]:
import torch
import cv2
import copy
import matplotlib.pyplot as plt
from  config.config import get_cfg
from model.rcnn import GeneralizedRCNN
from engine.optimizer import build_optimizer,build_lr_scheduler

from data.build import (
    build_detection_test_loader,
    build_detection_train_loader,
)
from data.mapper import DatasetMapper
import data.transforms as T
from data.phase_1 import load_voc_instances,VOC_CLASS_NAMES
from structures.image_list import ImageList
from engine.detection_checkpointer import DetectionCheckpointer
from data.utils import build_augmentation


In [3]:
cfg = get_cfg()
cfg.merge_from_file('./config_files/voc.yaml')
cfg.MODEL.SAVE_IDX = 13
cfg.MODEL.RPN.USE_MDN=False
cfg.MODEL.ROI_HEADS.USE_MLN=False
cfg.MODEL.RPN.AUTO_LABEL = True
cfg.MODEL.ROI_HEADS.AUTO_LABEL = False
cfg.log = False
cfg.MODEL.ROI_HEADS.AF = 'baseline'
NUM_CLASSES = 20
NUM_CLASSES += 1
cfg.INPUT.RANDOM_FLIP = "none"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = NUM_CLASSES
# cfg.merge_from_list(args.opts)
RPN_NAME = 'mdn' if cfg.MODEL.RPN.USE_MDN else 'base'
ROI_NAME = 'mln' if cfg.MODEL.ROI_HEADS.USE_MLN else 'base'
MODEL_NAME = RPN_NAME + ROI_NAME

In [ ]:
model = GeneralizedRCNN(cfg).to('cuda:0')
state_dict = torch.load('./ckpt/{}/{}_{}_5000.pt'.format(cfg.MODEL.ROI_HEADS.AF,cfg.MODEL.SAVE_IDX,MODEL_NAME))
model.load_state_dict(state_dict)
model.proposal_generator.SSL_MODEL.to('cuda:1')

In [5]:
DIR_NAME = '/data/private/OWOD/datasets/VOC2007'
split = 'train'
data = load_voc_instances(DIR_NAME,split,VOC_CLASS_NAMES,phase='t1',COCO_CLASS=False)
mapper = DatasetMapper(is_train=True, augmentations=build_augmentation(cfg,True))
data_loader = build_detection_train_loader(data,mapper=mapper,total_batch_size=1)
VOC_CLASS_NAMES_NEW = (*VOC_CLASS_NAMES, 'unknown')
loader = iter(data_loader)
it = 0

In [40]:
print(it)
batched_inputs = next(loader)
images = model.preprocess_image(batched_inputs)
features = model.backbone(images.tensor)
gt_instances = [x['instances'].to(model.device) for x in batched_inputs]
proposals, proposal_losses,gt_instances = model.proposal_generator(images, features, gt_instances)

image = batched_inputs[0]['image']
inputs = {"image": batched_inputs[0]['image'], 
            "height": batched_inputs[0]['height'], "width": batched_inputs[0]['width']}
# proposals = model._postprocess(gt_instances,[inputs],images.image_sizes)
height = inputs.get("height", images.image_sizes[0][0])
width = inputs.get("width", images.image_sizes[0][1])


new_size = (height, width)
output_width_tmp = width
output_height_tmp = height
scale_x, scale_y = (
        output_width_tmp / gt_instances[0].image_size[1],
        output_height_tmp / gt_instances[0].image_size[0],
    )
output_boxes = gt_instances[0].gt_boxes
output_boxes.scale(scale_x, scale_y)
output_boxes.clip(gt_instances[0].image_size)

file_name = batched_inputs[0]['file_name']
demo_image = cv2.imread(file_name)
boxes = output_boxes.tensor
labels = gt_instances[0].gt_classes
for label,bbox in zip(labels,boxes):
    if label == 20:
        color = (255,255,0)
    else:
        color = (0,255,0)
    cv2.rectangle(demo_image, (int(bbox[0]), int(bbox[1])), 
                            (int(bbox[2]),int(bbox[3])), color, 1)
demo_image = cv2.cvtColor(demo_image, cv2.COLOR_RGB2BGR)
plt.imshow(demo_image)
plt.savefig('./dummy/proposals/{}'.format(it))
plt.show()
it+=1

34
